# Regression with an Insurance Dataset
## Playground Series - Season 4, Episode 12
Submissions are evaluated using the Root Mean Squared Logarithmic Error (RMSLE).

In [1]:
from google.colab import drive
import sys

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)
sys.path.append('/content/drive/MyDrive')


Mounted at /content/drive


In [2]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import numpy as np
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer
!pip install scikit-optimize
!pip install shap
from skopt import BayesSearchCV
from skopt.space import Real, Integer
!pip install xgboost
import xgboost as xgb
from sklearn.model_selection import KFold
import shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.7 MB/s eta 0:00:00


## Load Data

In [3]:
train_set = pd.read_csv('/content/drive/MyDrive/Playground Series - Season 4, Episode 12/data/train.csv')
test_set = pd.read_csv('/content/drive/MyDrive/Playground Series - Season 4, Episode 12/data/test.csv')

train_set.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [4]:
train_set.describe()

,id,Age,Annual Income,Number of Dependents,Health Score,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Premium Amount
count,1.200000e+06,1.181295e+06,1.155051e+06,1.090328e+06,1.125924e+06,835971.000000,1.199994e+06,1.062118e+06,1.199999e+06,1.200000e+06
mean,5.999995e+05,4.114556e+01,3.274522e+04,2.009934e+00,2.561391e+01,1.002689,9.569889e+00,5.929244e+02,5.018219e+00,1.102545e+03
std,3.464103e+05,1.353995e+01,3.217951e+04,1.417338e+00,1.220346e+01,0.982840,5.776189e+00,1.499819e+02,2.594331e+00,8.649989e+02
min,0.000000e+00,1.800000e+01,1.000000e+00,0.000000e+00,2.012237e+00,0.000000,0.000000e+00,3.000000e+02,1.000000e+00,2.000000e+01
25%,2.999998e+05,3.000000e+01,8.001000e+03,1.000000e+00,1.591896e+01,0.000000,5.000000e+00,4.680000e+02,3.000000e+00,5.140000e+02
50%,5.999995e+05,4.100000e+01,2.391100e+04,2.000000e+00,2.457865e+01,1.000000,1.000000e+01,5.950000e+02,5.000000e+00,8.720000e+02
75%,8.999992e+05,5.300000e+01,4.463400e+04,3.000000e+00,3.452721e+01,2.000000,1.500000e+01,7.210000e+02,7.000000e+00,1.509000e+03
max,1.199999e+06,6.400000e+01,1.499970e+05,4.000000e+00,5.897591e+01,9.000000,1.900000e+01,8.490000e+02,9.000000e+00,4.999000e+03


In [5]:
for col in train_set.columns:
  if train_set[col].dtype == 'object':
    print(col, train_set[col].unique())

for col in train_set.columns:
    print(col, train_set[col].isna().sum())

for col in test_set.columns:
  if train_set[col].dtype == 'object':
    print(col, test_set[col].unique())

for col in test_set.columns:
    print(col, test_set[col].isna().sum())

Gender ['Female' 'Male']
Marital Status ['Married' 'Divorced' 'Single' nan]
Education Level ["Bachelor's" "Master's" 'High School' 'PhD']
Occupation ['Self-Employed' nan 'Employed' 'Unemployed']
Location ['Urban' 'Rural' 'Suburban']
Policy Type ['Premium' 'Comprehensive' 'Basic']
Policy Start Date ['2023-12-23 15:21:39.134960' '2023-06-12 15:21:39.111551'
 '2023-09-30 15:21:39.221386' ... '2021-04-28 15:21:39.129190'
 '2019-11-14 15:21:39.201446' '2020-10-19 15:21:39.118178']
Customer Feedback ['Poor' 'Average' 'Good' nan]
Smoking Status ['No' 'Yes']
Exercise Frequency ['Weekly' 'Monthly' 'Daily' 'Rarely']
Property Type ['House' 'Apartment' 'Condo']
id 0
Age 18705
Gender 0
Annual Income 44949
Marital Status 18529
Number of Dependents 109672
Education Level 0
Occupation 358075
Health Score 74076
Location 0
Policy Type 0
Previous Claims 364029
Vehicle Age 6
Credit Score 137882
Insurance Duration 1
Policy Start Date 0
Customer Feedback 77824
Smoking Status 0
Exercise Frequency 0
Property 

In [6]:
# Age
train_set['Age'] = train_set['Age']

def categorize_age(age):
    if age < 25:
        return '18-25'
    elif age < 40:
        return '25-40'
    elif age < 60:
        return '40-60'
    elif age >= 60:
        return '60+'
    else:
        return 'nan'


train_set['Age Cat'] = train_set['Age'].apply(categorize_age)
train_set = pd.get_dummies(train_set, columns=['Age Cat'], drop_first=False, dtype=float)
train_set['Age'] = train_set['Age'] / 100
train_set['Age'] = train_set['Age'].replace('nan', None).fillna(train_set['Age'].mode()[0])

# Gender
train_set['Gender'] = train_set['Gender'].replace({'Male': 0, 'Female': 1}).astype(int)

# Annual Income
train_set['Annual Income isna'] = train_set['Annual Income'].isna().astype(int)
train_set['Annual Income'] = train_set['Annual Income'].fillna(train_set['Annual Income'].mean())
train_set['Monthly Income'] = train_set['Annual Income'] / 12
train_set['Annual Income SQRT'] = np.sqrt(train_set['Annual Income'])
train_set['Annual Income Log'] = np.log(train_set['Annual Income'])
train_set['Annual Income SQ'] = train_set['Annual Income'] ** 2

# Marital Status
train_set['Marital Status'] = train_set['Marital Status'].fillna('nan')
train_set['Marital Status Int'] = train_set['Marital Status'].replace({'Married': 1, 'Divorced': 0.3, 'Single': 0.2, 'nan': 0}).astype(float)
train_set = pd.get_dummies(train_set, columns=['Marital Status'], drop_first=False, dtype=float)

# Number of Dependents
train_set['Number of Dependents isna'] = train_set['Number of Dependents'].isna().astype(int)
train_set['Number of Dependents'] = train_set['Number of Dependents'].fillna(0)
train_set['Number of Dependents'] = train_set['Number of Dependents'] / train_set['Number of Dependents'].max()

# Education Level
train_set['Education Level'] = train_set['Education Level'].fillna('nan')
train_set['Education'] = train_set['Education Level'].copy()
train_set = pd.get_dummies(train_set, columns=['Education'], drop_first=False, dtype=float)
train_set['Education Level'] = train_set['Education Level'].replace({'nan': -1, 'High School': 0, "Bachelor's": 1, "Master's": 2, 'PhD': 3}).astype(int)

# Occupation
train_set['Occupation'] = train_set['Occupation'].fillna('nan')
train_set = pd.get_dummies(train_set, columns=['Occupation'], drop_first=False, dtype=float)

# Health Score
train_set['Health Score'] = train_set['Health Score'].fillna(-100)
train_set['Health Score'] = train_set['Health Score'] / 100

# Location
train_set = pd.get_dummies(train_set, columns=['Location'], drop_first=False, dtype=float)

# Policy Type
train_set = pd.get_dummies(train_set, columns=['Policy Type'], drop_first=False, dtype=float)

# Previous Claims
train_set['Previous Claims Isna'] = train_set['Previous Claims'].isna().astype(int)
train_set['Previous Claims'] = train_set['Previous Claims'].fillna(-100)

# Vehicle Age
train_set['Vehicle Age'] = train_set['Vehicle Age'].fillna(train_set['Vehicle Age'].mode()[0])

# Policy Start Date
train_set['Policy Start Date'] = pd.to_datetime(train_set['Policy Start Date'])
train_set['Policy Start Date Year'] = train_set['Policy Start Date'].dt.year
train_set['Policy Start Date Month'] = train_set['Policy Start Date'].dt.month
train_set['Policy Start Date Day'] = train_set['Policy Start Date'].dt.day
train_set['Policy Start Date Weekday'] = train_set['Policy Start Date'].dt.weekday

# Smocking Status
train_set['Smoking Status'] = train_set['Smoking Status'].replace({'Yes': 1, 'No': 0}).astype(int)

# Exercise Frequency
train_set['Exercise Frequency'] = train_set['Exercise Frequency'].replace({'Daily': 1, 'Weekly': 0.5, 'Monthly': 0.3, 'Rarely': 0}).astype(float)

# Property Type
train_set['Property Type Int'] = train_set['Property Type'].replace({'Condo': 1, 'House': 0.4, 'Apartment': 0.2}).astype(float)
train_set = pd.get_dummies(train_set, columns=['Property Type'], drop_first=False, dtype=float)

# Customer Feedback
train_set['Customer Feedback'] = train_set['Customer Feedback'].fillna('nan')
train_set = pd.get_dummies(train_set, columns=['Customer Feedback'], drop_first=False, dtype=float)

# Credit Score
train_set['Credit Score isna'] = train_set['Credit Score'].isna().astype(int)
train_set['Credit Score'] = train_set['Credit Score'].fillna(train_set['Credit Score'].mean())

# Insurance Duration
train_set['Insurance Duration'] = train_set['Insurance Duration'].fillna(train_set['Insurance Duration'].mode()[0])

train_set.drop(columns=['Policy Start Date'], inplace=True)

In [7]:
# Age
test_set['Age'] = test_set['Age']

def categorize_age(age):
    if age < 25:
        return '18-25'
    elif age < 40:
        return '25-40'
    elif age < 60:
        return '40-60'
    elif age >= 60:
        return '60+'
    else:
        return 'nan'


test_set['Age Cat'] = test_set['Age'].apply(categorize_age)
test_set = pd.get_dummies(test_set, columns=['Age Cat'], drop_first=False, dtype=float)
test_set['Age'] = test_set['Age'] / 100
test_set['Age'] = test_set['Age'].replace('nan', None).fillna(train_set['Age'].mode()[0])

# Gender
test_set['Gender'] = test_set['Gender'].replace({'Male': 0, 'Female': 1}).astype(int)

# Annual Income
test_set['Annual Income isna'] = test_set['Annual Income'].isna().astype(int)
test_set['Annual Income'] = test_set['Annual Income'].fillna(train_set['Annual Income'].mean())
test_set['Monthly Income'] = test_set['Annual Income'] / 12
test_set['Annual Income SQRT'] = np.sqrt(test_set['Annual Income'])
test_set['Annual Income Log'] = np.log(test_set['Annual Income'])
test_set['Annual Income SQ'] = test_set['Annual Income'] ** 2

# Marital Status
test_set['Marital Status'] = test_set['Marital Status'].fillna('nan')
test_set['Marital Status Int'] = test_set['Marital Status'].replace({'Married': 1, 'Divorced': 0.3, 'Single': 0.2, 'nan': 0}).astype(float)
test_set = pd.get_dummies(test_set, columns=['Marital Status'], drop_first=False, dtype=float)

# Number of Dependents
test_set['Number of Dependents isna'] = test_set['Number of Dependents'].isna().astype(int)
test_set['Number of Dependents'] = test_set['Number of Dependents'].fillna(0)
test_set['Number of Dependents'] = test_set['Number of Dependents'] / test_set['Number of Dependents'].max()

# Education Level
test_set['Education Level'] = test_set['Education Level'].fillna('nan')
test_set['Education'] = test_set['Education Level'].copy()
test_set = pd.get_dummies(test_set, columns=['Education'], drop_first=False, dtype=float)
test_set['Education Level'] = test_set['Education Level'].replace({'nan': -1, 'High School': 0, "Bachelor's": 1, "Master's": 2, 'PhD': 3}).astype(int)

# Occupation
test_set['Occupation'] = test_set['Occupation'].fillna('nan')
test_set = pd.get_dummies(test_set, columns=['Occupation'], drop_first=False, dtype=float)

# Health Score
test_set['Health Score'] = test_set['Health Score'].fillna(-100)
test_set['Health Score'] = test_set['Health Score'] / 100

# Location
test_set = pd.get_dummies(test_set, columns=['Location'], drop_first=False, dtype=float)

# Policy Type
test_set = pd.get_dummies(test_set, columns=['Policy Type'], drop_first=False, dtype=float)

# Previous Claims
test_set['Previous Claims Isna'] = test_set['Previous Claims'].isna().astype(int)
test_set['Previous Claims'] = test_set['Previous Claims'].fillna(-100)

# Vehicle Age
test_set['Vehicle Age'] = test_set['Vehicle Age'].fillna(train_set['Vehicle Age'].mode()[0])

# Policy Start Date
test_set['Policy Start Date'] = pd.to_datetime(test_set['Policy Start Date'])
test_set['Policy Start Date Year'] = test_set['Policy Start Date'].dt.year
test_set['Policy Start Date Month'] = test_set['Policy Start Date'].dt.month
test_set['Policy Start Date Day'] = test_set['Policy Start Date'].dt.day
test_set['Policy Start Date Weekday'] = test_set['Policy Start Date'].dt.weekday

# Smocking Status
test_set['Smoking Status'] = test_set['Smoking Status'].replace({'Yes': 1, 'No': 0}).astype(int)

# Exercise Frequency
test_set['Exercise Frequency'] = test_set['Exercise Frequency'].replace({'Daily': 1, 'Weekly': 0.5, 'Monthly': 0.3, 'Rarely': 0}).astype(float)

# Property Type
test_set['Property Type Int'] = test_set['Property Type'].replace({'Condo': 1, 'House': 0.4, 'Apartment': 0.2}).astype(float)
test_set = pd.get_dummies(test_set, columns=['Property Type'], drop_first=False, dtype=float)

# Customer Feedback
test_set['Customer Feedback'] = test_set['Customer Feedback'].fillna('nan')
test_set = pd.get_dummies(test_set, columns=['Customer Feedback'], drop_first=False, dtype=float)

# Credit Score
test_set['Credit Score isna'] = test_set['Credit Score'].isna().astype(int)
test_set['Credit Score'] = test_set['Credit Score'].fillna(train_set['Credit Score'].mean())

# Insurance Duration
test_set['Insurance Duration'] = test_set['Insurance Duration'].fillna(train_set['Insurance Duration'].mode()[0])

test_set.drop(columns=['Policy Start Date'], inplace=True)

In [8]:
train_set.info()
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 58 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   id                         1200000 non-null  int64  
 1   Age                        1200000 non-null  float64
 2   Gender                     1200000 non-null  int64  
 3   Annual Income              1200000 non-null  float64
 4   Number of Dependents       1200000 non-null  float64
 5   Education Level            1200000 non-null  int64  
 6   Health Score               1200000 non-null  float64
 7   Previous Claims            1200000 non-null  float64
 8   Vehicle Age                1200000 non-null  float64
 9   Credit Score               1200000 non-null  float64
 10  Insurance Duration         1200000 non-null  float64
 11  Smoking Status             1200000 non-null  int64  
 12  Exercise Frequency         1200000 non-null  float64
 13  Premium Amou

In [9]:
id_train=train_set['id']
y_train = train_set['Premium Amount']
X_train = train_set.drop(columns=['id', 'Premium Amount'])

id_test=test_set['id']
X_test = test_set.drop(columns=['id'])

# XGB Regression

In [11]:
# Log-transform the target variable
y_train_log = np.log(y_train)

# Define parameter search space
search_spaces = {
    'learning_rate': Real(0.001, 0.5, 'uniform'),
    'max_depth': Integer(1, 14),
    'n_estimators': Integer(50, 300),
}

# Initialize the XGBRegressor model with GPU support
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    tree_method = 'hist',
    device = 'cuda',
    verbosity=2
)

kf = KFold(n_splits=5)  # 10-fold cross-validation

# Using negative RMSE as the scoring metric
optimizer = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=32,
    cv=kf,
    scoring='neg_root_mean_squared_error',  # RMSE scoring
    verbose=1,
    n_jobs=-1,
    return_train_score=True
)

# Fit the optimizer using X_train and log-transformed y_train
optimizer.fit(X_train, y_train_log)

# Get the best model
best_model = optimizer.best_estimator_

# Save the best model
model_path = '/content/drive/MyDrive/Playground Series - Season 4, Episode 12/models/XGB_1.json'
best_model.save_model(model_path)

# Get cross-validation errors
cv_results = optimizer.cv_results_

# Calculate and print the mean RMSE for each fold (convert negative RMSE back to positive)
mean_rmse = -cv_results['mean_test_score']  # This is negative RMSE, so we negate it to get RMSE

print(f"Best parameters: {optimizer.best_params_}")
print(f"Mean Cross-validation RMSE: {mean_rmse.mean():.4f}")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:14:23] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [14:14:23] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)


Best parameters: OrderedDict([('learning_rate', 0.05104644623518532), ('max_depth', 7), ('n_estimators', 133)])
Mean Cross-validation RMSE: 1.0745


In [16]:
from google.colab import files
# Convert test data to DMatrix
dtest = xgb.DMatrix(X_test)

# Load the trained model
model_path = f'/content/drive/MyDrive/Playground Series - Season 4, Episode 12/models/XGB_1.json'
model = xgb.Booster(model_file=model_path)

dtrain = xgb.DMatrix(X_train)
y_hat_train = model.predict(dtrain)
y_hat_train = np.exp(y_hat_train)
print(' Root Mean Squared Logarithmic Error', np.sqrt(np.mean((np.log(y_train) - np.log(y_hat_train))**2)))

# Get predictions
preds_log = model.predict(dtest)

# Convert log-transformed predictions back to original scale
preds_xgb = np.exp(preds_log)

# Ensure the length of predictions matches the number of rows in the test data
assert len(preds_xgb) == len(id_test), "Mismatch between number of predictions and test data IDs"

# Create output DataFrame with original Id and predicted SalePrice
output = pd.DataFrame({'id': id_test, 'Premium Amount': preds_xgb.squeeze()})

# Remove any duplicate rows by 'Id'
output.drop_duplicates(subset='id', keep='first', inplace=True)

# Save predictions to a CSV file
output.to_csv('XGB_predictions.csv', index=False)
files.download('XGB_predictions.csv')

 Root Mean Squared Logarithmic Error 1.0474368532908842


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Lasso Regression

In [17]:
from sklearn import linear_model
# Log-transform the target variable
y_train_log = np.log(y_train)
clf = linear_model.Lasso(alpha=0.65, max_iter=5000, copy_X= True)
clf.fit(X_train,y_train_log)

y_hat_train = clf.predict(X_train)
y_hat_train = np.exp(y_hat_train)

print(' Root Mean Squared Logarithmic Error', np.sqrt(np.mean((np.log(y_train) - np.log(y_hat_train))**2)))

predictions = clf.predict(X_test)

# Convert log-transformed predictions back to original scale
Lasso_preds = np.exp(predictions)

# Create output DataFrame with original Id and predicted SalePrice
output = pd.DataFrame({'id': id_test, 'Premium Amount': Lasso_preds.squeeze()})

# Remove any duplicate rows by 'Id'
output.drop_duplicates(subset='id', keep='first', inplace=True)

# Save predictions to a CSV file
output.to_csv('Lasso_predictions.csv', index=False)
files.download('Lasso_predictions.csv')

 Root Mean Squared Logarithmic Error 1.0950782808903086


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Linear Regression

In [18]:
from sklearn import linear_model
# Log-transform the target variable
y_train_log = np.log(y_train)
lr = linear_model.LinearRegression(copy_X= True, n_jobs=-1)
lr.fit(X_train,y_train_log)

y_hat_train = lr.predict(X_train)
y_hat_train = np.exp(y_hat_train)

print(' Root Mean Squared Logarithmic Error', np.sqrt(np.mean((np.log(y_train) - np.log(y_hat_train))**2)))

predictions = lr.predict(X_test)

# Convert log-transformed predictions back to original scale
LR_preds = np.exp(predictions)

# Create output DataFrame with original Id and predicted SalePrice
output = pd.DataFrame({'id': id_test, 'Premium Amount': LR_preds.squeeze()})

# Remove any duplicate rows by 'Id'
output.drop_duplicates(subset='id', keep='first', inplace=True)

# Save predictions to a CSV file
output.to_csv('LR_predictions.csv', index=False)
files.download('LR_predictions.csv')

 Root Mean Squared Logarithmic Error 1.0854711138727149


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Weighted Average of the predictions based on the Train Error

In [20]:
output = pd.DataFrame({'id': id_test, 'Premium Amount 1': preds_xgb.squeeze(), 'Premium Amount 2': Lasso_preds.squeeze(), 'Premium Amount 3': LR_preds.squeeze()})
output['Premium Amount'] = output.apply(lambda row: (row['Premium Amount 1']*5 + row['Premium Amount 2'] + row['Premium Amount 3']*2) / 8, axis=1)
output.drop(columns=['Premium Amount 1', 'Premium Amount 2', 'Premium Amount 3'], inplace=True)

# Save predictions to a CSV file
output.to_csv('Average_predictions.csv', index=False)
files.download('Average_predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>